In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/convolve-epoch1/sample_submission.csv
/kaggle/input/convolve-epoch1/train.json
/kaggle/input/convolve-epoch1/test.csv


In [2]:
import json
with open("/kaggle/input/convolve-epoch1/train.json","r") as train_csv:
    train_data=json.load(train_csv)
X=[]
y=[]
for key in train_data:
    X.append(key)
    y.append(train_data[key])

In [3]:
for i in range(len(X)):
    X[i]=str(X[i]).strip().strip("\n").split(" ",5)
    
df=pd.DataFrame(X,columns=["ID1","Date","device_ID","DateTime","ID3","message"])
df["label"]=y
#convert to datetime format
df["DateTime"]=pd.to_datetime(df["DateTime"],format=("%Y-%m-%d-%H.%M.%S.%f"))
df.head()

,ID1,Date,device_ID,DateTime,ID3,message,label
0,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.363779,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...,normal
1,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.527847,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...,normal
2,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.675872,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...,normal
3,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.823719,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...,normal
4,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.982731,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...,normal


In [4]:
df = df[df["device_ID"]!="-"]

In [5]:
df.drop(["Date","ID3","ID1","device_ID","DateTime"],axis=1,inplace=True)

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
import re
import nltk
from nltk.corpus import stopwords

In [7]:
STOPWORDS=set(stopwords.words("english"))
def clean_text(text):
    text=text.lower()
    text=re.sub(r'[^a-zA-Z]', " ", text)
    text=re.sub(r'\s+'," ",text)
    text=" ".join(word for word in text.split() if word not in STOPWORDS)
    return text

In [8]:
df["clean_text"]=df["message"].apply(clean_text)
X=df["clean_text"]
y=df["label"]

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=True, stratify=y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=50, shuffle=True, stratify=y)
# model training

from sklearn.svm import SVC
model = SVC(C=5)

pip_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', model)])
pip_model.fit(X_train, y_train)
    
print('Accuracy:', pip_model.score(X_test, y_test)*100)


#print("CV Score:", np.mean(cv_score)*100)
y_pred = pip_model.predict(X_test)

Accuracy: 99.99985550427563


In [11]:
df_submission=pd.read_csv("/kaggle/input/convolve-epoch1/test.csv")

In [12]:
df_submission["clean_text"]=df_submission[" Log"].apply(clean_text)
y_sub = pip_model.predict(df_submission["clean_text"])
df_sub=pd.read_csv("/kaggle/input/convolve-epoch1/sample_submission.csv")
df_sub[" Label"]=y_sub

In [13]:
df_sub.to_csv("submission.csv", index=False)